In [2]:
$Paths = @{ 
    CloneRoot = 'G:\2023-git\Pwsh📁'
}
function InvokeNativeCommand { 
    <#
    .synopsis
    slighly safer/mainly sugar for invoking native commands
    .notes
        Warning, if you are running on ~7.2, there's a breaking change that you may need to use this setting
        $PSNativeCommandArgumentPassing = 'Legacy'
    .example
        InvokeNativeCommand -name 'ping' -args @('google.com', '/all')
    #>

    [OutputType('System.Management.Automation.ApplicationInfo', 'System.Management.Automation.CommandInfo')]
    [Alias('InvokeCmd')]
    param( 
        [Alias('Name')]
        [Parameter(Mandatory, Position= 0)]
        [ArgumentCompletions('git', 'gh', 'pwsh')]
        [string]$CommandName,
        
        [Alias('Args')][object[]]$ArgumentList,
        
        # don't actually invoke, just preview the commandline
        [Parameter()]
        [Alias('WhatIf')]
        [switch]$TestOnly,

        # If you want the Command itself, instead of running it
        [Alias('WithoutInvoke')]
        [Parameter()][switch]$PassThru
     )
    # or $ExecutionContext.InvokeCommand.GetCommand('gh', 'Application')
    $bin = gcm -name $CommandName -CommandType Application -ea 'stop' -totalCount 1    
    "invoke => {0}`n    {1}" -f @( 
        $bin
        $ArgumentList -join ' '
    ) | write-host -fore 'darkgreen'

    # Warning, if you are running on ~7.2, there's a breaking change to native commands that you may need to use this setting
    if( $PSNativeCommandArgumentPassing ) { $PSNativeCommandArgumentPassing = 'Legacy' }

    if($PassThru) { return $bin }
    if($TestOnly) { return } 

    & $bin @Argumentlist    
}
function GhClone {
    # minimal wrapper of 'gh' clone 
    param( 
        # ex: 'StartAutomating/ugit'
        [ArgumentCompletions(            
            'PowerShell/PowerShell', 'dfinke/ImportExcel', 'StartAutomating/ugit')]
        [string]$OwnerRepoName,

        # if not set, 'gh' defaults to using the 'repoName' without 'OwnerName'
        [string]$Destination,

        [Alias('WhatIf')][switch]$TestOnly
    )
    [Collections.Generic.List[Object]]$ghArgs = @(
        'repo', 'clone', $OwnerRepoName, $Destination    
    )

    if(-not ($PSBoundParameters.ContainsKey('Destination'))) {
        $destination = $OwnerRepoName -split '/' | select -last 1 
    }

    'cloning {0} => to {1}' -f @( 
        $OwnerRepoName
        $Destination
    ) | write-host -fore 'blue'

    pushd -stack 'gh' $Paths.CloneRoot
    if(Test-path ($destination)) {
        "Repo already exists: '$Destination'"
        return
    }
    
    InvokeNativeCommand -TestOnly:$TestOnly -CommandName 'gh' -ArgumentList @(
        'repo', 'clone'
        $OwnerRepoName 
        if($Destination) { $Destination }
    )
    popd -stack 'gh'
}


In [6]:
# InvokeNativeCommand -name 'ping' -args @('google.com', '/all') -TestOnly
pushd -stack 'gh' $Paths.CloneRoot
GhClone -OwnerRepoName 'StartAutomating/ugit' -TestOnly
GhClone -OwnerRepoName 'PowerShell/PowerShell' -Dest 'Pwsh' -TestOnly
popd -stack 'gh'

# actual invoke
pushd -stack 'gh' $Paths.CloneRoot
GhClone -OwnerRepoName 'StartAutomating/ugit'
GhClone -OwnerRepoName 'PowerShell/PowerShell' -Dest 'Pwsh'
gci
popd -stack 'gh'
gci

cloning StartAutomating/ugit => to ugit
invoke => gh.exe
    repo clone StartAutomating/ugit ugit
cloning PowerShell/PowerShell => to Pwsh
invoke => gh.exe
    repo clone PowerShell/PowerShell Pwsh
cloning StartAutomating/ugit => to ugit
invoke => gh.exe
    repo clone StartAutomating/ugit ugit
fatal: destination path 'ugit' already exists and is not an empty directory.
failed to run git: exit status 128
cloning PowerShell/PowerShell => to Pwsh
invoke => gh.exe
    repo clone PowerShell/PowerShell Pwsh
fatal: destination path 'Pwsh' already exists and is not an empty directory.
failed to run git: exit status 128

    Directory: G:\2023-git\Pwsh📁

Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
d----           5/16/2023 10:46 AM                JustinGrote🧑
d----           5/30/2023  4:11 PM                Pwsh
d----           5/30/2023  4:45 PM                ugit

    Directory: H:\data\2023\pwsh\notebooks\Pwsh\Native.Comman